### Spark application (instance of SparkContext)

Application
  
  - a driver program 
  - executors on the cluster.
   (parallel operations)

### The main abstraction:
1. resilient distributed dataset (RDD),

2. shared variables

 - broadcast variables
 - accumulators
 
### Deploy mode	
Distinguishes where the driver process runs.
- In "cluster" mode, the framework launches the driver inside of the cluster. 
- In "client" mode, the submitter launches the driver outside of the cluster.



# Linking with Spark

In [ ]:
bin/spark-submit 
#script located in the Spark directory. This script will load Spark’s Java/Scala libraries 
#and allow you to submit applications to a cluster. 

bin/pyspark
#to launch an interactive Python shell.

from pyspark import SparkContext, SparkConf
#import some Spark classes into your program

$ PYSPARK_PYTHON=python3.4 bin/pyspark
$ PYSPARK_PYTHON=/opt/pypy-2.5/bin/pypy bin/spark-submit examples/src/main/python/pi.py
# It uses the default python version in PATH, you can specify which version of Python you want to use by 
# PYSPARK_PYTHON

# Initializing Spark

In [ ]:
# The first thing a Spark program must do is to create a SparkContext object, 
# which tells Spark how to access a cluster.
conf = SparkConf().setAppName(appName).setMaster(master)
sc = SparkContext(conf=conf)
 

# To create a SparkContext you first need to build a SparkConf object that contains 
# information about your application.

???
# master is a Spark, Mesos or YARN cluster URL, or a special “local” string to run in 
#local mode. 
#In practice, when running on a cluster, you will not want to hardcode master in the 
#program, but rather launch the application with spark-submit and receive it there

# Using the Shell--PySpark shell

In [ ]:
In the PySpark shell, a special interpreter-aware SparkContext is already created for you,
in the variable called sc.

In [ ]:
#For example, to run bin/pyspark on exactly four cores,

$ ./bin/pyspark --master local[4] --py-files code.py

# bin/pyspark
#to launch an interactive Python shell.
# --master argument: 
#which master the context connects to using
# --py-files
#add Python .zip, .egg or .py files to the runtime path by passing a comma-separated list


In [ ]:
pyspark --help
#for help

#launch the PySpark shell in IPython
#To use IPython, set the PYSPARK_DRIVER_PYTHON variable to ipython when running bin/pyspark:
$ PYSPARK_DRIVER_PYTHON=ipython ./bin/pyspark

#To use the Jupyter notebook (previously known as the IPython notebook)
$ PYSPARK_DRIVER_PYTHON=jupyter PYSPARK_DRIVER_PYTHON_OPTS=notebook ./bin/pyspark

# （input the command %pylab inline as part of your notebook before you start to try Spark 
# from the Jupyter notebook.）

# Resilient Distributed Datasets (RDDs)
a fault-tolerant collection of elements that can be operated on in parallel. 

two ways to create RDDs: 
 - parallelizing an existing collection in your driver program, 
 - or referencing a dataset in an external storage system:
    
    such as a shared filesystem, HDFS, HBase, or any data source offering a Hadoop InputFormat.

### External Datasets

In [ ]:
#Text file RDDs can be created using SparkContext’s textFile method. 
#This method takes an URI for the file (either a local path on the machine, or 
#a hdfs://, s3n://, etc URI) and reads it as a collection of lines.
distFile = sc.textFile("data.txt")

#add up the sizes of all the lines using the map and reduce operations
distFile.map(lambda s: len(s)).reduce(lambda a, b: a + b)

### Parallelized Collections

In [ ]:
#For example, here is how to create a parallelized collection holding the numbers 1 to 5:

data = [1, 2, 3, 4, 5]
distData = sc.parallelize(data,10)
# the number of partitions: 10
#One important parameter for parallel collections is the number of partitions to cut the 
#dataset into. 
#Spark will run one task for each partition of the cluster. 
#Typically you want 2-4 partitions for each CPU in your cluster.

# add up the elements of the list.
distData.reduce(lambda a, b: a + b)

### RDD Operations
 - transformations:
   
   which create a new dataset from an existing one
   
   
 - actions (“jobs” (Spark actions) )
  
  which return a value to the driver program after running a computation on the dataset. 

In [ ]:
#defines a base RDD from an external file
lines = sc.textFile("data.txt")

#defines lineLengths as the result of a map transformation(lazy).
lineLengths = lines.map(lambda s: len(s))

#lineLengths.persist()
#cause lineLengths to be saved in memory after the first time it is computed.

#run reduce, which is an action. At this point Spark breaks the computation into tasks 
#to run on separate machines, and each machine runs both its part of the map and a 
#local reduction, returning only its answer to the driver program.
totalLength = lineLengths.reduce(lambda a, b: a + b)

### Passing Functions to Spark
- Lambda expressions
- Local defs inside the function calling into Spark, for longer code.
- Top-level functions in a module.

In [ ]:
"""MyScript.py"""
if __name__ == "__main__":
    def myFunc(s):
        words = s.split(" ")
        return len(words)

    sc = SparkContext(...)
    sc.textFile("file.txt").map(myFunc)

# Submitting Applications

In [1]:
The spark-submit script in Spark’s bin directory 

SyntaxError: invalid syntax (<ipython-input-1-07b91afb836c>, line 1)

### Bundling Your Application’s Dependencies

In [2]:
create an assembly jar (or “uber” jar) containing your code and its dependencies

For Python, you can use the 
--py-files argument of spark-submit 
to add .py, .zip or .egg files to be distributed with your application. 
If you depend on multiple Python files we recommend packaging them into a .zip or .egg.

SyntaxError: invalid syntax (<ipython-input-2-326a7989e57a>, line 1)

### launch the application with spark-submit

In [3]:
./bin/spark-submit \
  --class <main-class> \
  --master <master-url> \
  --deploy-mode <deploy-mode> \
  --conf <key>=<value> \
  ... # other options
  <application-jar> \
  [application-arguments]

SyntaxError: invalid syntax (<ipython-input-3-333507bd1494>, line 1)

# Monitoring
Every SparkContext launches a web UI, by default on port 4040

In [ ]:
http://<driver-node>:4040
        

# Job Scheduling